### IMPORTS

In [18]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

pd.get_option("display.max_columns")

20

### READ DATA

In [19]:
anon_prices = pd.read_csv("anonymized_transactions.csv")

### DATA PREP

In [20]:
anon_prices[['transaction_date', 'product_level_1', 'product_level_2', 'zproduct_id']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_date  999 non-null    object
 1   product_level_1   999 non-null    object
 2   product_level_2   999 non-null    object
 3   zproduct_id       999 non-null    object
dtypes: object(4)
memory usage: 31.3+ KB


In [21]:
anon_prices['transaction_month'] = pd.to_datetime(anon_prices['transaction_date']).dt.month

In [22]:
anon_prices[['transaction_month', 'product_level_1', 'product_level_2', 'zproduct_id']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   transaction_month  999 non-null    int64 
 1   product_level_1    999 non-null    object
 2   product_level_2    999 non-null    object
 3   zproduct_id        999 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.3+ KB


In [23]:
anon_prices[ 'transaction_month'].value_counts()

3     282
1     271
12    231
2     215
Name: transaction_month, dtype: int64

In [24]:
anon_prices.describe()

,sm_transaction_0002_key,anchor_weight,test_field_2,targetpricetag,startpricetag,revenue,quantity,profit,price_unit,price_normalizer,...,lee_spend_bin,test_me_bin,snk_spend_bin,cv_split_1,target_nv,start_nv,floor_nv,cv_split_3,cv_split_2,transaction_month
count,0.0,0.0,0.0,0.0,0.0,999.000000,999.000000,999.000000,999.000000,999.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.000000
mean,NaN,NaN,NaN,NaN,NaN,2754.158627,244.094285,735.029055,11.547703,8.207517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.323323
std,NaN,NaN,NaN,NaN,NaN,5691.460171,406.173816,1524.541181,5.555201,3.790929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.277518
min,NaN,NaN,NaN,NaN,NaN,0.000000,2.000000,-4268.482000,0.000000,1.715260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,607.381150,50.000000,134.671550,7.515714,5.439222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,1369.218900,100.000000,326.965500,10.689700,7.396065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
75%,NaN,NaN,NaN,NaN,NaN,2900.379250,200.000000,824.422350,14.766741,10.336887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
max,NaN,NaN,NaN,NaN,NaN,76633.801900,4000.000000,19461.298000,69.912030,37.312920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000


In [25]:
anon_prices.head(3)

,sm_transaction_0002_key,transaction_id,anchor_weight,transaction_date,test_field_2,targetpricetag,startpricetag,revenue,region_group,quantity,...,lee_spend_bin,test_me_bin,snk_spend_bin,cv_split_1,target_nv,start_nv,floor_nv,cv_split_3,cv_split_2,transaction_month
0,NaN,2-039995466:25,NaN,2019-12-03 00:00:00.000000,NaN,NaN,NaN,2924.7128,B9,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
1,NaN,2-039997979:21,NaN,2019-12-05 00:00:00.000000,NaN,NaN,NaN,26427.2451,SK,3000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,NaN,2-040000029:11,NaN,2019-12-09 00:00:00.000000,NaN,NaN,NaN,4188.5119,WLS,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12


### Data Prep: Bayesian Hierarchical Regression

In [26]:
df_params = {"hbr_b_mu": -0.25,
             "hbr_b_sig": 0.05,
             "Test_Time_Index": [12, 13],
             "Min_Elast": 0.5,
             "Max_Elast": 2.5}

hbr_b_mu = float(df_params['hbr_b_mu'])
hbr_b_sig = float(df_params['hbr_b_sig'])

anon_prices['MCMC_Partition'] = 'Global_Partition'

MCMC_Partition = np.unique(anon_prices['MCMC_Partition'])

bayes_hier = anon_prices[['MCMC_Partition',
                          'product_level_1',
                          'product_level_2',
                          'zproduct_id',
                          'transaction_month',
                          'quantity',
                          'revenue']]

bayes_tree = bayes_hier[['product_level_1',
                         'product_level_2',
                         'zproduct_id']].columns
tree_levels = len(bayes_tree)

### Metrics: Bayesian Hierarchical Regression: Statistics

In [27]:
groupby_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month']
quantity_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month', 'quantity']
revenue_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month', 'revenue']

bayes_quantity = bayes_hier[quantity_list].groupby(by=groupby_list).sum()
bayes_revenue = bayes_hier[revenue_list].groupby(by=groupby_list).sum()
bayes_avg_price = bayes_revenue['revenue'] / bayes_quantity['quantity']

bayesXmean = bayes_avg_price.mean()
bayesXstd = bayes_avg_price.std()

bayesYmean = bayes_quantity['quantity'].mean()
bayesYstd = bayes_quantity['quantity'].std()

bayesX_zscore =  (bayes_avg_price - bayesXmean) / bayesXstd
bayesY_zscore =  (bayes_quantity - bayesYmean) / bayesYstd

bayesX_zscore = bayesX_zscore.to_numpy()
bayesY_zscore = bayesY_zscore['quantity'].to_numpy()

### Metrics: Bayesian Hierarchical Regression: Levels

In [28]:
new_bayes = bayes_hier[['MCMC_Partition', 'product_level_1', 'product_level_2', 'zproduct_id']]
sort_by_list1 = ['product_level_1']
sort_by_list2 = ['product_level_1', 'product_level_2']
sort_by_list3 = ['product_level_1', 'product_level_2', 'zproduct_id']

In [29]:
# level 1
new_bayes['level1']= new_bayes.sort_values(by=sort_by_list1)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

# level 2
new_bayes['level2'] = new_bayes.sort_values(by=sort_by_list2)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

# level 3
new_bayes['level3'] = new_bayes.sort_values(by=sort_by_list3)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

<ipython-input-29-3fe17a380076>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_bayes['level1']= new_bayes.sort_values(by=sort_by_list1)\
<ipython-input-29-3fe17a380076>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_bayes['level2'] = new_bayes.sort_values(by=sort_by_list2)\


In [30]:
level_1 = new_bayes['level1'].to_numpy()
level_1_unique = np.unique(level_1)
level_1_count = len(level_1_unique)

level_2 = new_bayes['level2'].to_numpy()
level_2_link = new_bayes[['level1', 'level2']].copy().drop_duplicates()['level1'].to_numpy()
level_2_unique = np.unique(level_2)
level_2_count = len(level_2_unique)

level_3 = new_bayes['level3'].to_numpy()
level_3_link = new_bayes[['level2', 'level3']].copy().drop_duplicates()['level2'].to_numpy()
level_3_unique = np.unique(level_3)
level_3_count = len(level_3_unique)

### PROGRAM

In [42]:
with pm.Model() as hierarchical_model:

    b_0 = pm.Normal(name='b_0', mu=hbr_b_mu, sigma=hbr_b_sig)

    b_1 = pm.Normal(name='b_1', mu=b_0,  sigma=hbr_b_sig, shape=level_1_count)

    b_2 = pm.Normal(name='b_2', mu=b_1[level_2_link - 1],  sigma=hbr_b_sig, shape=level_2_count)

    b_3 = pm.Normal(name='b_3', mu=b_2[level_3_link - 1],  sigma=hbr_b_sig, shape=level_3_count)

    eps = pm.HalfCauchy(name='eps', beta=1)

    # Likelihood
    Y_est = b_3[level_3[:689] - 1] * bayesX_zscore
    Y_like = pm.Normal(name='Y_like', mu=Y_est, sigma=eps, observed=bayesY_zscore)

    trace = pm.sample(draws=2000, tune=2000, cores=4, chains=2, random_seed=1, progressbar=0)

In [43]:
az.summary(data=trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b_0,-0.118,0.031,-0.172,-0.058,0.010,0.007,11.0,18.0,1.17
b_1[0],-0.119,0.059,-0.234,-0.002,0.009,0.006,44.0,220.0,1.04
b_1[1],-0.117,0.057,-0.230,-0.006,0.009,0.006,42.0,318.0,1.05
b_1[2],-0.117,0.059,-0.237,-0.007,0.009,0.006,45.0,351.0,1.04
b_1[3],-0.117,0.059,-0.231,-0.000,0.008,0.006,53.0,184.0,1.03
...,...,...,...,...,...,...,...,...,...
b_3[995],-0.120,0.092,-0.299,0.061,0.009,0.006,103.0,894.0,1.02
b_3[996],-0.120,0.091,-0.297,0.056,0.007,0.005,166.0,1206.0,1.02
b_3[997],-0.119,0.092,-0.294,0.066,0.007,0.005,190.0,1658.0,1.01
b_3[998],-0.119,0.092,-0.291,0.063,0.006,0.004,251.0,1271.0,1.01


In [58]:
print(trace['posterior']['b_3'].mean(axis=1).shape)
trace

(2, 999)


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

In [56]:
trace['posterior']['b_3'].mean(axis=1)

<xarray.DataArray 'b_3' (chain: 2, b_3_dim_0: 999)>
array([[-0.12920171, -0.12870248, -0.12682539, ..., -0.12906995,
        -0.12855815, -0.1270994 ],
       [-0.10869975, -0.11042147, -0.10956097, ..., -0.11036843,
        -0.10910808, -0.11060258]])
Coordinates:
  * chain      (chain) int64 0 1
  * b_3_dim_0  (b_3_dim_0) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998

In [57]:
trace['posterior']['b_3'].mean(axis=1).mean(axis=1)

<xarray.DataArray 'b_3' (chain: 2)>
array([-0.12704732, -0.10800505])
Coordinates:
  * chain    (chain) int64 0 1

In [55]:
trace['posterior']['b_3'].mean(axis=1).mean(axis=1).mean(axis=0)

<xarray.DataArray 'b_3' ()>
array(-0.11752618)

In [75]:
b_est = trace['posterior']['b_3'].mean(axis=1).mean(axis=1).mean(axis=0).to_numpy()

a_est_0 = bayesYmean - ((b_est * bayesXmean * bayesYstd) / bayesXstd)
b_est_0 = (b_est * bayesYstd) / bayesXstd

y_est = a_est_0 + (b_est_0 * bayes_avg_price.to_numpy())
mape = np.mean(np.abs((y_est - bayes_quantity.to_numpy()) / bayes_quantity.to_numpy()))

## PLOT
# prediction_vs_actual_graph = np.sum(bayes_avg_price.to_numpy()) vs np.sum(bayes_quantity.to_numpy()) and np.sum(y_est)

elasticity = -1 * (b_est_0 * bayes_avg_price.to_numpy()) / (a_est_0 + b_est_0 * bayes_avg_price.to_numpy())

In [76]:
mape

3.471947525828108

In [80]:
len(elasticity), elasticity

(689,
 array([ 0.48732389,  0.13309704,  0.77348002,  0.33608317,  0.43943468,
         0.29753876,  0.61560583,  0.42315462,  0.42315462,  0.55507073,
         0.13114778,  0.20920744,  0.50338611,  0.27722172,  0.27722175,
         0.34250458,  0.16974645,  0.37886972,  0.08036122,  0.77477885,
         0.8591515 ,  0.71777193,  0.81645605,  0.61897605,  6.94198726,
         0.25375746,  0.24237918,  0.53702823,  0.33745118,  0.26956432,
         0.31106641,  0.32257437,  0.43657766,  0.20882378,  0.22072088,
         0.7513853 ,  0.51695504,  0.51695491,  0.30384967,  0.75883088,
         0.29638138,  0.1647114 ,  0.33719403,  0.60858115,  0.80546597,
         0.57395275,  0.34613322,  0.42936617,  0.24852577,  0.22273424,
         0.18403314,  0.13597428,  0.15934209,  0.29306275,  0.29306275,
         0.24199828,  0.42511116,  0.1445241 ,  0.16073997,  0.46684071,
         0.59418042,  1.84994758,  0.67990545,  0.10069378,  0.22067912,
         0.54783897,  0.54783896,  0.30991865